# Обучение мультиклассовой модели на архитектуре DistilBERT для предсказания услуг по страховым письмам

---



#### Разделы ноутбука

1. [Импорт библиотек Python и подготовка среды](#section01)
2. [Импорт и предварительная обработка данных](#section02)
3. [Подготовка набора данных и загрузчика данных](#section03)
4. [Создание нейронной сети для точной настройки](#section04)
5. [Точная настройка модели](#section05)
6. [Проверка работоспособности модели](#section06)
7. [Сохранение модели для будущего использования](#section07)

#### Техническая информация

- Каждая строка обучающего датасета содержит:
	- ID письма (letter_id)
	- Текст письма (guarantee_letter_text)
	- Коды услуг (service_ids_list)
	- Признак наличия услуг (class)


Каждое письмо может быть одновременно отнесено к нескольким услугам. В этом случае в поле "Коды услуг" будет перечислено несколько кодов, например `[1, 2, 3]`

 - Используемая языковая модель:
	 - DistilBERT это модель-трансформер меньшего размера по сравнению с BERT или Roberta.  
	 - [Пример использования](https://medium.com/huggingface/distilbert-8cf3380435b5)
	 - [Научная статья](https://arxiv.org/pdf/1910.01108)
     - [Документация](https://huggingface.co/transformers/model_doc/distilbert.html)


 - Системные требования:
	 - Python 3.6 или выше
	 - Pytorch, Transformers и все стандартные библиотеки Python ML
	 - доступная GPU

<a id='section01'></a>
### Импорт библиотек Python и подготовка среды

На этом шаге мы импортируем библиотеки и модули, необходимые для запуска нашего скрипта. Список библиотек:
* warnings
* Numpy
* Pandas
* tqdm
* scikit-learn metrics
* Pytorch
* Pytorch Utils for Dataset and Dataloader
* Transformers
* DistilBERT Model and Tokenizer
* logging

После этого мы подготовим устройство к выполнению CUDA. Эта конфигурация необходима, если вы хотите использовать встроенный графический процессор.

In [ ]:
# ! pip install transformers==3.0.2

In [ ]:
# Импорт стандартных библиотек для ML
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel, AutoTokenizer, AutoModel, BertTokenizer, BertModel
import logging
import ast
from distilbert_class import DistilBERTClass
from bert_multiclass_classifier import BertMulticlassClassifier
logging.basicConfig(level=logging.ERROR)

In [ ]:
# Проверка использования GPU
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
# Метод для расчета метрики Хэмминга
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

<a id='section02'></a>
### Импорт и предварительная обработка данных

Мы будем работать с данными и подготавливать их к точной настройке модели.

* На первом шаге мы удаляем **id** письма из датасета.
* Создается новый датафрейм с колонкой **text**, в которую добавляется код услуги по справочнику.
* Значения всех кодов услуг преобразуются в массив.
* Массив кодов преобразуется в массив меток (0 или 1) для каждой услуги в колонке **labels**.

In [ ]:
data = pd.read_csv('/datasets/multiclass_df.csv')
data.head()

,letter_id,guarantee_letter_text,service_ids_list,class
0,0,Исключить интракраниальное объемное образовани...,"[32, 18]",1
1,1,( G93.4 ) Диагноз : Вестибулоатактический синд...,"[32, 18]",1
2,2,( G93.4 ) Ингосстрах Магнитно-резонансная томо...,"[32, 18]",1
3,3,"( G93.4 ) Оплата медицинских услуг , оказанных...","[32, 18]",1
4,4,"Оплата медицинских услуг , оказанных по данном...","[32, 18]",1


In [ ]:
data.drop(['letter_id'], inplace=True, axis=1)

In [ ]:
# Общие параметры обучения
MAX_LEN = 512
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 25
LEARNING_RATE = 1e-05
NUM_CLASSES = 75
tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny')

In [ ]:
# Метод преобразует строковое представление кодов услуг в массив one-hot encoding из 75 элементов
# Например, строка "[1, 4, 5]" будет преобразована в: [0, 1, 0, 0, 1, 1, 0, ...] 
def str_to_list(string):
  if string[0] != '[':
    string = '[' + string + ']'

  arr = ast.literal_eval(string)
  new_arr = []
  for i in range(NUM_CLASSES):
    if i in arr or str(i) in arr:
      new_arr.append(1)
    else:
      new_arr.append(0)

  return new_arr

In [ ]:
# Сформируем датасет для обучения из двух колонок
new_df = pd.DataFrame()
new_df['text'] = data['guarantee_letter_text']
new_df['labels'] = data['service_ids_list'].apply(str_to_list)

In [ ]:
new_df.head(3)

,text,labels
0,Исключить интракраниальное объемное образовани...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,( G93.4 ) Диагноз : Вестибулоатактический синд...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,( G93.4 ) Ингосстрах Магнитно-резонансная томо...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


<a id='section03'></a>
### Подготовка набора данных и загрузчика данных

Мы начнем с определения нескольких ключевых переменных, которые будут использоваться позже на этапе обучения/точной настройки.
Затем следует создание класса набора данных с несколькими метками - это определяет, как текст предварительно обрабатывается перед отправкой его в нейронную сеть. Мы также определим загрузчик данных (Dataloader), который будет передавать данные пакетами в нейронную сеть для соответствующего обучения и обработки.
Dataset и Dataloader - это конструкции библиотеки PyTorch для определения и управления предварительной обработкой данных и их передачей в нейронную сеть. Более подробно можно ознакомиться здесь [Документация PyTorch](https://pytorch.org/docs/stable/data.html)

#### *MultiLabelDataset* Dataset Class
- Этот класс принимает `tokenizer`, `dataframe` и `max_length` в качестве входных данных и генерирует токенизированные выходные данные и теги, которые используются моделью BERT для обучения.
- Мы используем токенизатор DistilBERT для токенизации данных в столбце `text` датафрейма.
- Токенизатор использует метод `encode_plus` для выполнения токенизации и генерации необходимых выходных данных: `ids`, `attention_mask`, `token_type_ids`

- Подробнее ознакомиться с токенизатором можно [по ссылке](https://huggingface.co/transformers/model_doc/distilbert.html#distilberttokenizer)
- `targets` это список услуг, обозначенных `0` или `1` в датафрейме.
- Класс *MultiLabelDataset* создает 2 датасета, для обучения и валидации.
- *Обучающий Датасет* используется для точной настройки модели: **80% от исходного датасета**
- *Валидационный Датасет* используется для оценки производительности модели. Модель не видела эти данные во время обучения.

#### Dataloader
- Dataloader используется для создания обучающего и валидационного загрузчика данных, который загружает данные в нейронную сеть определенным образом. Это необходимо, поскольку все данные из набора данных не могут быть загружены в память сразу, следовательно, необходимо контролировать объем данных, загружаемых в память и затем передаваемых в нейронную сеть.
- Этот контроль достигается с помощью таких параметров, как `batch_size` и `max_len`.
- Загрузчики обучающих и валидационных данных используются в обучающей и валидационной частях соответственно

In [ ]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
# Создание датасета и загрузчика данных
train_size = 0.8
train_data=new_df.sample(frac=train_size,random_state=200)
test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

FULL Dataset: (29715, 2)
TRAIN Dataset: (23772, 2)
TEST Dataset: (5943, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

<a id='section04'></a>
### Создание нейронной сети для точной настройки

#### Нейронная сеть
 - Мы будем создавать нейронную сеть с помощью `DistilBERTClass`.
 - Эта сеть будет основана на модели `DistilBERT`.  За ней идут слои `Dropout` и `Linear`. Они добавлены с целью **Регуляризации** и **Классификации** соответственно.
 - В прямом цикле есть 2 выхода из слоя `DistilBERTClass`.
 - Второй выход `output_1` или по-другому `pooled output` передается в слой `Dropout`, а последующий выходной сигнал в слой `Linear`.
 - Обратите внимание, что количество измерений для слоя `Linear` равно **75**, потому что это общее количество услуг, по которым мы хотим классифицировать нашу модель.
 - Данные будут переданы в `DistilBERTClass`, как определено в наборе данных.
 - Выходные данные конечного слоя - это то, что будет использоваться для расчета потерь и определения точности прогнозирования моделей.
 - Мы создадим экземпляр сети под названием `model`. Этот экземпляр будет использоваться для обучения, а затем для сохранения окончательной обученной модели для будущего вывода.

#### Функция потерь и оптимизатор
 - Функция потерь определена далее как `loss_fn`.
 - Используемая функция потерь будет представлять собой комбинацию двоичной перекрестной энтропии, которая реализована следующим образом [BCELogits Loss](https://pytorch.org/docs/stable/nn.html#bcewithlogitsloss) в PyTorch
 - `Optimizer` определен ниже.
 - `Optimizer` используется для обновления весов нейронной сети с целью повышения ее производительности.

In [ ]:
# Создаем модель, добавляя слои drop out и полносвязный поверх distil bert, чтобы получить финальные данные от модели.
class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.l1 = BertModel.from_pretrained("cointegrated/rubert-tiny")
        self.pre_classifier = torch.nn.Linear(312, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, NUM_CLASSES)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids, attention_mask, token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = DistilBERTClass()
model.to(device)

In [ ]:
# Метод, вычисляющий функцию потерь
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
# Будем использовать оптимизатор Adam для обучения
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

<a id='section05'></a>
### Точная настройка модели

Здесь мы определяем обучающую функцию, которая обучает модель на обучающем наборе данных, созданном выше, указанное количество раз (эпох). Эпоха определяет, сколько раз полные данные будут переданы через сеть.

В этой функции происходят следующие события для точной настройки нейронной сети:
- Загрузчик данных передает данные в модель на основе размера пакета.
- Последующий вывод из модели и фактическая категория сравниваются для расчета потерь.
- Значение потерь используется для оптимизации весов нейронов в сети.
- После каждых 5000 шагов значение потерь выводится в консоль.

In [ ]:
def train(epoch):
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        loss.backward()
        optimizer.step()

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Epoch: 0, Loss:  0.6967678070068359


743it [02:43,  4.55it/s]
2it [00:00,  4.99it/s]

Epoch: 1, Loss:  0.16762596368789673


743it [02:47,  4.43it/s]
2it [00:00,  5.16it/s]

...

743it [02:47,  4.44it/s]
1it [00:00,  3.50it/s]

Epoch: 23, Loss:  0.0047399671748280525


743it [02:46,  4.45it/s]
2it [00:00,  5.15it/s]

Epoch: 24, Loss:  0.0027604231145232916


743it [02:46,  4.45it/s]


<a id='section06'></a>
### Проверка работоспособности модели

На этапе проверки мы передаем невидимые данные (тестовый набор данных) в модель. Этот шаг определяет, насколько хорошо модель работает с невидимыми данными.

Эти невидимые данные представляют собой 20% файла `multiclass_df.csv`, который был отделен на этапе создания набора данных.
На этапе проверки веса модели не обновляются. Только конечный результат сравнивается с фактическим значением. Это сравнение затем используется для расчета точности модели.

Как сказано выше, чтобы получить оценку производительности наших моделей, мы используем следующие показатели:
- Метрика Хэмминга
- Функция потерь Хэмминга


In [ ]:
def validation(testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
outputs, targets = validation(testing_loader)

final_outputs = np.array(outputs) >= 0.5

0it [00:00, ?it/s]

torch.Size([32, 512])


In [ ]:
val_hamming_loss = metrics.hamming_loss(targets, final_outputs)
val_hamming_score = hamming_score(np.array(targets), np.array(final_outputs))

print(f"Hamming Score = {val_hamming_score}")
print(f"Hamming Loss = {val_hamming_loss}")

Hamming Score = 0.994166806887655
Hamming Loss = 0.00012788154130910314


<a id='section07'></a>
### Сохранение модели для будущего использования

Это заключительный шаг в процессе точной настройки модели.

Модель и ее словарный запас сохраняются локально. Эти файлы затем используются в будущем для определения новых предоставляемых услуг.

In [ ]:
# Сохраним файлы модели для использования в будущем

output_model_file = './models/bert_multiclass.pt'
output_vocab_file = './models/bert_multiclass_vocab.pt'

torch.save(model, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('Saved')

Saved
